## PDF MCP Server 測試

測試 pdf_server.py - 一個可以讀取和搜尋 PDF 內容的 MCP Server

### 功能
- `get_pdf_info`: 取得 PDF 基本資訊（頁數、作者等）
- `extract_pdf_text`: 抽取 PDF 文字內容
- `search_pdf`: 在 PDF 中搜尋關鍵字

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

### 1. 列出 PDF Server 的 Tools

In [ ]:
params = {"command": "uv", "args": ["run", "pdf_server.py"]}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    tools = await server.list_tools()
    print("Available tools:")
    for tool in tools:
        print(f"\n📄 {tool.name}")
        print(f"   {tool.description}")

### 2. 準備測試用 PDF

請確保你有一個 PDF 檔案可以測試。修改下方的 `pdf_path` 變數。

In [ ]:
# 修改這裡指向你的 PDF 檔案
pdf_path = "/path/to/your/file.pdf"

# 或者使用專案中的 PDF（如果有的話）
# pdf_path = "../assets/sample.pdf"

### 3. 讓 Agent 讀取 PDF 資訊

In [ ]:
instructions = """
You are a PDF assistant that can read and analyze PDF documents.
You can:
- Get PDF information (pages, author, title)
- Extract text from specific pages or entire documents
- Search for keywords in PDF documents
"""

request = f"Please tell me about this PDF file: {pdf_path}. How many pages does it have?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="pdf_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("pdf_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 4. 讓 Agent 抽取 PDF 內容

In [ ]:
request = f"Extract the text from the first page of {pdf_path}"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="pdf_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("pdf_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 5. 讓 Agent 搜尋 PDF 內容

In [ ]:
keyword = "your_keyword"  # 修改成你要搜尋的關鍵字
request = f"Search for '{keyword}' in {pdf_path} and show me where it appears."

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="pdf_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("pdf_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 6. 綜合問答 - 讓 Agent 分析 PDF

In [ ]:
request = f"""
Please analyze {pdf_path}:
1. Tell me the basic info (pages, title, author)
2. Summarize what the first page is about
"""

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="pdf_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("pdf_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 7. 直接測試 Python（不透過 MCP）

In [ ]:
import pypdf

# 直接使用 pypdf
try:
    with open(pdf_path, "rb") as f:
        reader = pypdf.PdfReader(f)
        print(f"Pages: {len(reader.pages)}")
        print(f"Title: {reader.metadata.title if reader.metadata else 'N/A'}")
        print(f"\nFirst page preview:")
        print(reader.pages[0].extract_text()[:500])
except FileNotFoundError:
    print(f"File not found: {pdf_path}")
except Exception as e:
    print(f"Error: {e}")